# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
pip install gmaps


     |████████████████████████████████| 1.1 MB 3.7 MB/s eta 0:00:01
     |████████████████████████████████| 121 kB 52.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 119.1 MB/s eta 0:00:01
     |████████████████████████████████| 243 kB 103.1 MB/s eta 0:00:01


  Created wheel for gmaps: filename=gmaps-0.9.0-py2.py3-none-any.whl size=2076097 sha256=cca645a6a3f9a00faaa6deefb52817d96ba110d06e69e69a825b4c7519f6d6cb
  Stored in directory: /Users/marcellisv/Library/Caches/pip/wheels/30/bd/87/d0d731ee712c34a4d6bace8b1dbf8c8932e22a1625b4a55be4
Successfully built gmaps
Note: you may need to restart the kernel to use updated packages.


In [132]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [133]:
cities_data = pd.read_csv("cities.csv")
cities_data.dropna()
cities_data.head()

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Carnarvon,40,AU,1614742236,83,-24.8667,113.6333,78.8,13.80
1,1,Tasiilaq,90,GL,1614742264,39,65.6145,-37.6368,26.6,18.41
2,2,Safford,1,US,1614742372,15,32.8340,-109.7076,57.2,11.50
3,3,La Ronge,91,CA,1614742340,79,55.1001,-105.2842,15.8,5.75
4,4,Punta Arenas,20,CL,1614742218,66,-53.1500,-70.9167,48.2,11.50


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [134]:
gmaps.configure(g_key)

In [135]:
locations = cities_data [["Lat", "Lng"]]
Humidity = cities_data["Humidity"].astype(float)

In [136]:
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

heat_layer = gmaps.heatmap_layer (locations, weights=Humidity,
                                 dissipating=False, max_intensity=100)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [137]:
hotel_df = cities_data.loc[(cities_data['Max Temp'] >= 70) & (cities_data ['Max Temp'] <= 80) & (cities_data ['Cloudiness']==0) & (cities_data ['Wind Speed'] < 10)]
hotel_df

,City ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
99,99,Morón,0,AR,1614742281,89,-34.6534,-58.6198,79.00,4.00
124,124,Saint-Philippe,0,RE,1614742236,61,-21.3585,55.7679,77.00,4.61
133,133,Panabá,0,MX,1614742579,78,21.2833,-88.2667,72.32,5.41
178,178,Vaijāpur,0,IN,1614742585,29,19.9167,74.7333,77.00,9.75
253,253,Lázaro Cárdenas,0,MX,1614742594,83,17.9583,-102.2000,71.33,3.29
273,273,Cartagena,0,CO,1614742597,83,10.3997,-75.5144,77.00,8.05
307,307,Maceió,0,BR,1614742603,94,-9.6658,-35.7353,73.40,2.30
318,318,Balrāmpur,0,IN,1614742604,17,27.4333,82.1833,73.22,5.35
319,319,Saint-Pierre,0,RE,1614742285,61,-21.3393,55.4781,77.00,4.61
329,329,Alta Floresta,0,BR,1614742605,100,-9.8756,-56.0861,73.40,1.14


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [138]:
hotel_df["Hotel Name"]=""

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key,
    "location": "23.61,58.59"
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
hotel_name = requests.get(base_url, params=params)
    
hotel_name = hotel_name.json()

hotel_name

<ipython-input-138-b9d5d281ddd8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""


{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 23.5671512, 'lng': 58.6155072},
    'viewport': {'northeast': {'lat': 23.56829382989272,
      'lng': 58.61675992989272},
     'southwest': {'lat': 23.56559417010727, 'lng': 58.61406027010727}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'Al Bustan Palace, A Ritz-Carlton Hotel',
   'opening_hours': {'open_now': True},
   'photos': [{'height': 576,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/108593968168219378701">Al Bustan Palace, a Ritz-Carlton Hotel</a>'],
     'photo_reference': 'ATtYBwJCv0CMBzUbn0kEtabUjPQSBU-kHDfueSh7TvSOF0fUlOMQBNAjlaMHlmPg3KrcvNp1rNolethg8eQL4nXwKvxUDsPfHo6MCzrIC9Ml9i1heq13IjTFKFMbWxFK-pg1yW99vvKRfI7Z2rpJ529rV4cH_cx83lK581VNTD0BePBcKdSd',
     'width': 1024}],
   'place_id': 'ChIJgUYW6zrxkT4RIaJvt4wvA_Q',
   'plus_code': {'compound_code': 'HJ88+V6 Muscat, Oman',
    '

In [139]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [140]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))